In [28]:
import os
basepath = os.path.abspath(os.path.join(os.path.dirname('__file__'), '../..'))
basepath

'c:\\Users\\wwxc9\\Desktop\\git\\LLM_task_practice'

# 文本分类实例

## Step1 导入相关包

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

## Step2 加载数据集

In [23]:
dataset = load_dataset("csv", data_files=f"{basepath}/datasets/ChnSentiCorp_htl_all.csv", split="train")
dataset = dataset.filter(lambda x: x["review"] is not None)
dataset

Dataset({
    features: ['label', 'review'],
    num_rows: 7765
})

## Step3 划分数据集

In [24]:
datasets = dataset.train_test_split(test_size=0.1)
datasets

DatasetDict({
    train: Dataset({
        features: ['label', 'review'],
        num_rows: 6988
    })
    test: Dataset({
        features: ['label', 'review'],
        num_rows: 777
    })
})

## Step4 数据集预处理

In [27]:
import torch

tokenizer = AutoTokenizer.from_pretrained("hfl/rbt3")

def process_function(examples):
    tokenized_examples = tokenizer(examples["review"], max_length=128, truncation=True)
    tokenized_examples["labels"] = examples["label"]
    return tokenized_examples

tokenized_datasets = datasets.map(process_function, batched=True, remove_columns=datasets["train"].column_names)
tokenized_datasets

Map:   0%|          | 0/777 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 6988
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 777
    })
})

## Step5 创建模型

In [5]:
model = AutoModelForSequenceClassification.from_pretrained("hfl/rbt3")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hfl/rbt3 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
model.config

BertConfig {
  "_name_or_path": "hfl/rbt3",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 3,
  "output_past": true,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.41.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 21128
}

## Step6 创建评估函数

In [22]:
import evaluate

acc_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

In [8]:
def eval_metric(eval_predict):
    predictions, labels = eval_predict
    predictions = predictions.argmax(axis=-1)
    acc = acc_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels)
    acc.update(f1)
    return acc

## Step7 创建TrainingArguments

In [9]:
train_args = TrainingArguments(
    output_dir="./checkpoints",      # 输出文件夹
    per_device_train_batch_size=64,  # 训练时的batch_size
    per_device_eval_batch_size=128,  # 验证时的batch_size
    logging_steps=10,                # log 打印的频率
    evaluation_strategy="epoch",     # 评估策略
    save_strategy="epoch",           # 保存策略
    save_total_limit=3,              # 最大保存数
    learning_rate=2e-5,              # 学习率
    weight_decay=0.01,               # weight_decay
    metric_for_best_model="f1",      # 设定评估指标
    load_best_model_at_end=True      # 训练完成后加载最优模型
)
train_args

d:\Programmer\Anaconda3\envs\transformers39\lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=2,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_steps=None,
eval_strategy=epoch,
evaluation_strategy=epoch,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_c

## Step8 创建Trainer

In [10]:
from transformers import DataCollatorWithPadding
trainer = Trainer(
    model=model, 
    args=train_args, 
    train_dataset=tokenized_datasets["train"], 
    eval_dataset=tokenized_datasets["test"], 
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=eval_metric
)

## Step9 模型训练

In [11]:
trainer.train()

  0%|          | 0/330 [00:00<?, ?it/s]

{'loss': 0.6124, 'grad_norm': 3.754300117492676, 'learning_rate': 1.9393939393939395e-05, 'epoch': 0.09}
{'loss': 0.5517, 'grad_norm': 1.7708921432495117, 'learning_rate': 1.8787878787878792e-05, 'epoch': 0.18}
{'loss': 0.4525, 'grad_norm': 2.2372066974639893, 'learning_rate': 1.8181818181818182e-05, 'epoch': 0.27}
{'loss': 0.415, 'grad_norm': 2.968520402908325, 'learning_rate': 1.7575757575757576e-05, 'epoch': 0.36}
{'loss': 0.3316, 'grad_norm': 2.7214412689208984, 'learning_rate': 1.6969696969696972e-05, 'epoch': 0.45}
{'loss': 0.3526, 'grad_norm': 3.5977938175201416, 'learning_rate': 1.6363636363636366e-05, 'epoch': 0.55}
{'loss': 0.3706, 'grad_norm': 2.7457494735717773, 'learning_rate': 1.575757575757576e-05, 'epoch': 0.64}
{'loss': 0.2914, 'grad_norm': 2.2017719745635986, 'learning_rate': 1.5151515151515153e-05, 'epoch': 0.73}
{'loss': 0.3341, 'grad_norm': 2.876446485519409, 'learning_rate': 1.4545454545454546e-05, 'epoch': 0.82}
{'loss': 0.3, 'grad_norm': 4.467002868652344, 'lear

  0%|          | 0/7 [00:00<?, ?it/s]

{'eval_loss': 0.281263142824173, 'eval_accuracy': 0.8828828828828829, 'eval_f1': 0.9124157844080847, 'eval_runtime': 1.6623, 'eval_samples_per_second': 467.424, 'eval_steps_per_second': 4.211, 'epoch': 1.0}
{'loss': 0.3091, 'grad_norm': 2.576425790786743, 'learning_rate': 1.2727272727272728e-05, 'epoch': 1.09}
{'loss': 0.2407, 'grad_norm': 2.177828550338745, 'learning_rate': 1.2121212121212122e-05, 'epoch': 1.18}
{'loss': 0.2405, 'grad_norm': 1.9559743404388428, 'learning_rate': 1.1515151515151517e-05, 'epoch': 1.27}
{'loss': 0.2612, 'grad_norm': 2.114827871322632, 'learning_rate': 1.0909090909090909e-05, 'epoch': 1.36}
{'loss': 0.2613, 'grad_norm': 4.5404181480407715, 'learning_rate': 1.0303030303030304e-05, 'epoch': 1.45}
{'loss': 0.2537, 'grad_norm': 2.1063425540924072, 'learning_rate': 9.696969696969698e-06, 'epoch': 1.55}
{'loss': 0.2626, 'grad_norm': 2.6620242595672607, 'learning_rate': 9.090909090909091e-06, 'epoch': 1.64}
{'loss': 0.2746, 'grad_norm': 2.777951955795288, 'learni

  0%|          | 0/7 [00:00<?, ?it/s]

{'eval_loss': 0.2659030258655548, 'eval_accuracy': 0.8828828828828829, 'eval_f1': 0.9117361784675073, 'eval_runtime': 1.3405, 'eval_samples_per_second': 579.619, 'eval_steps_per_second': 5.222, 'epoch': 2.0}
{'loss': 0.2068, 'grad_norm': 1.968543529510498, 'learning_rate': 6.060606060606061e-06, 'epoch': 2.09}
{'loss': 0.2086, 'grad_norm': 5.35244083404541, 'learning_rate': 5.4545454545454545e-06, 'epoch': 2.18}
{'loss': 0.2053, 'grad_norm': 1.828860878944397, 'learning_rate': 4.848484848484849e-06, 'epoch': 2.27}
{'loss': 0.2298, 'grad_norm': 3.416945695877075, 'learning_rate': 4.242424242424243e-06, 'epoch': 2.36}
{'loss': 0.269, 'grad_norm': 3.1943516731262207, 'learning_rate': 3.6363636363636366e-06, 'epoch': 2.45}
{'loss': 0.2183, 'grad_norm': 5.594645023345947, 'learning_rate': 3.0303030303030305e-06, 'epoch': 2.55}
{'loss': 0.2141, 'grad_norm': 2.7982661724090576, 'learning_rate': 2.4242424242424244e-06, 'epoch': 2.64}
{'loss': 0.2355, 'grad_norm': 4.990346431732178, 'learning_r

  0%|          | 0/7 [00:00<?, ?it/s]

{'eval_loss': 0.2675485908985138, 'eval_accuracy': 0.8815958815958816, 'eval_f1': 0.9122137404580153, 'eval_runtime': 1.349, 'eval_samples_per_second': 575.969, 'eval_steps_per_second': 5.189, 'epoch': 3.0}
{'train_runtime': 138.6507, 'train_samples_per_second': 151.2, 'train_steps_per_second': 2.38, 'train_loss': 0.293150359211546, 'epoch': 3.0}


TrainOutput(global_step=330, training_loss=0.293150359211546, metrics={'train_runtime': 138.6507, 'train_samples_per_second': 151.2, 'train_steps_per_second': 2.38, 'total_flos': 351909933963264.0, 'train_loss': 0.293150359211546, 'epoch': 3.0})

## Step10 模型评估

In [12]:
trainer.evaluate(tokenized_datasets["test"])

  0%|          | 0/7 [00:00<?, ?it/s]

{'eval_loss': 0.281263142824173,
 'eval_accuracy': 0.8828828828828829,
 'eval_f1': 0.9124157844080847,
 'eval_runtime': 1.3812,
 'eval_samples_per_second': 562.542,
 'eval_steps_per_second': 5.068,
 'epoch': 3.0}

## Step11 模型预测

In [13]:
trainer.predict(tokenized_datasets["test"])

  0%|          | 0/7 [00:00<?, ?it/s]

PredictionOutput(predictions=array([[ 0.49870867, -1.0974436 ],
       [ 1.3614624 , -1.5930262 ],
       [ 1.3349482 , -1.6656973 ],
       ...,
       [-0.79037684,  0.07163587],
       [-1.3992751 ,  1.8859935 ],
       [-0.19881225,  0.39807355]], dtype=float32), label_ids=array([0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1,
       1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1,
       0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0,
       0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1,
       0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1

In [14]:
from transformers import pipeline

id2_label = id2_label = {0: "差评！", 1: "好评！"}
model.config.id2label = id2_label
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)

In [15]:
sen = "我觉得不错！"
pipe(sen)

[{'label': '好评！', 'score': 0.9660133719444275}]